# VOLCANA


**Autores:** [Jesús Javier Apesteguía, Mónica Marín, Joel Reyes, Laura Rodríguez]  
**Institución:** IPNA-CSIC  
**Fechas del proyecto:** [09/03/2025 – 09/05/2025]

Este proyecto tiene como objetivo analizar el comportamiento térmico de los volcanes El Teide, La Palma y Timanfaya (Lanzarote) mediante datos satelitales, con el fin de detectar patrones estacionales o posibles signos de actividad volcánica.

## Objetivos

- Descargar y procesar datos satelitales de radianza.
- Convertir datos de radianza a temperatura de brillo (BT).
- Calcular las imágenes de referencia mensuales (REF) a partir de la temperatura de brillo.
- Calcular la Potencia Radiactiva (FRP) diaria en áreas volcánicas.
- Evaluar variaciones térmicas y posibles correlaciones con actividad sísmica.

## Estructura de Carpetas

- `A00_data/`
  - `B_raw/`: Datos descargados sin procesar.
  - `B_processed/`: Datos procesados (BT, REF, FRP).
- `A01_source/`: Scripts de procesamiento y análisis.
- `A02_utils/`: Funciones auxiliares y herramientas reutilizables.
- `A03_web/`: Recursos relacionados con la interfaz web del proyecto
- `A04_tests/`: Scripts unitarios para validación del código.
- `A05_notebooks/`: Documentación formal del proyecto.

In [2]:
# Código para mostrar cómo se descargan los datos

import sys
from pathlib import Path

# Añadir la raíz del proyecto al path (ajusta el nombre según tu estructura)
project_root = next(p for p in Path().resolve().parents if p.name == "Practicas_Empresa_CSIC-1")
sys.path.append(str(project_root))

from A01_source.B01_1_download.La_Palma.download_LaPalma import descargar_datos1

descargar_datos1()

Ruta de salida: /Users/moni/Desktop/Practicas_Empresa_CSIC-1/A00_data/B_raw/La_Palma/2025_124

📅 Downloading data for 2025-124...
🔍 Searching for files of VJ102IMG...
📥 Downloading VJ102IMG.A2025124.0000.021.2025124064120.nc...
⚠️ Error processing VJ102IMG.A2025124.0000.021.2025124064120.nc: [Errno -101] NetCDF: HDF error: '/Users/moni/Desktop/Practicas_Empresa_CSIC-1/A00_data/B_raw/La_Palma/2025_124/VJ102IMG.A2025124.0000.021.2025124064120.nc'
📥 Downloading VJ102IMG.A2025124.0006.021.2025124064050.nc...
❌ Does not meet conditions. Deleting...
📥 Downloading VJ102IMG.A2025124.0012.021.2025124064800.nc...
❌ Does not meet conditions. Deleting...
📥 Downloading VJ102IMG.A2025124.0018.021.2025124064049.nc...
❌ Does not meet conditions. Deleting...
📥 Downloading VJ102IMG.A2025124.0024.021.2025124064054.nc...
❌ Does not meet conditions. Deleting...
📥 Downloading VJ102IMG.A2025124.0030.021.2025124064049.nc...
❌ Does not meet conditions. Deleting...
📥 Downloading VJ102IMG.A2025124.0036.021.20251

## Cálculo de la Temperatura de Brillo (BT)

Este proceso convierte archivos satelitales diarios del producto `VJ102IMG` en mapas de **temperatura de brillo (BT)** sobre la isla de La Palma.

---

### 1. Configuración de rutas
- Se detecta automáticamente la raíz del proyecto `Practicas_Empresa_CSIC`.
- Se establece la carpeta de entrada:  
  `A00_data/B_raw/La_Palma/`  
  con archivos NetCDF descargados cada día.
- Se define la carpeta de salida:  
  `A00_data/B_processed/La_Palma/BT_daily_pixels/`  
  donde se almacenan los resultados procesados.

---

### 2.Cálculo de la Temperatura de Brillo (BT)

La temperatura de brillo se calcula a partir de la radiancia usando la fórmula inversa de Planck:

\[
BT = \frac{c_2}{\lambda \cdot \ln\left( \frac{c_1}{L \cdot \lambda^5} + 1 \right)}
\]

donde:

- \( L \): radiancia medida  
- \( \lambda = 11.45\,\mu m \)  
- \( c_1 = 1.191042 \times 10^8 \)  
- \( c_2 = 1.4387752 \times 10^4 \)

---

### 3. Extracción de coordenadas
- A partir de los atributos globales (`NorthBoundingCoordinate`, `SouthBoundingCoordinate`, etc.), se construye una malla de latitudes y longitudes.

---

### 4. Guardado de resultados
- Se crea un `xarray.DataArray` con dimensiones (`time`, `y`, `x`) y coordenadas georreferenciadas (`latitude`, `longitude`).
- Si existe un archivo mensual, se acumulan los nuevos datos en él.
- Se aplica compresión (`zlib`) y se guarda en formato NetCDF con tipo `float32`.

---

### 5. Resultado final
- El archivo generado tiene el nombre: BT_La_Palma_VJ102IMG_YYYYMMDD.nc
  

In [26]:
import runpy
from pathlib import Path
import os

# Localiza manualmente la raíz del proyecto (ajusta este nombre si es diferente)
for parent in Path().resolve().parents:
    if "Practicas_Empresa_CSIC" in parent.name:
        project_root = parent
        break
else:
    raise RuntimeError("No se encontró la carpeta del proyecto en la ruta actual.")

# Ruta al script original
script_path = project_root / "A01_source" / "B01_3_processing" / "La_Palma" / "BT" / "BT_auto.py"

# Establece la variable de entorno __file__ para que el script funcione
os.environ["__file__"] = str(script_path)

# Ejecuta el script y evita mostrar su salida
_ = runpy.run_path(str(script_path), run_name="__main__")

StopIteration: 

## Generación del archivo REF mensual (La Palma)

Este script calcula la **temperatura de brillo media mensual (REF)** filtrada espacialmente sobre la región volcánica de La Palma. A continuación, se describen los pasos clave que realiza:

1. **Configuración de rutas:**
   - Detecta automáticamente la raíz del proyecto (`Practicas_Empresa_CSIC`).
   - Define el directorio de entrada (`BT_daily_pixels`) y de salida (`REF`).

2. **Región geográfica de interés:**
   - Se selecciona un área específica en La Palma entre las latitudes 28.55–28.65 y longitudes -17.93 a -17.80.

3. **Carga del archivo mensual de BT:**
   - Abre el archivo `BT_LaPalma_VJ102IMG_YYYY_MM.nc` correspondiente al mes actual.
   - Verifica que contiene la variable `BT_I05`.

4. **Procesamiento de cada escena diaria:**
   - Para cada imagen del mes:
     - Se establece el sistema de coordenadas geográficas (`EPSG:4326`).
     - Se reproyecta para unificar dimensiones.
     - Se aplica una máscara geográfica para recortar el área del volcán.
     - Se filtra la escena.

5. **Cálculo del REF mensual:**
   - Se promedian todas las escenas válidas del mes para obtener una imagen final.
   - Se guarda como NetCDF (`brightness_temperature_REF`), con metadatos y lista de escenas usadas.

6. **Salida:**
   - El archivo final se guarda en:  
     `A00_data/B_processed/La_Palma/REF/Ref_YYYY_MM.nc`

> Este proceso asegura que solo se utilicen escenas con información térmica significativa y de buena calidad para construir un mapa representativo de referencia mensual (REF).

In [17]:
from pathlib import Path

# Localiza la raíz del proyecto
project_root = next(p for p in Path().resolve().parents if "Practicas_Empresa_CSIC" in p.name)

# Ruta al script que genera la REF
script_path = project_root / "A01_source" / "B01_3_processing" / "La_Palma" / "REF" / "REF_auto.py"

# Ejecutar el script de forma segura, definiendo __file__
with open(script_path) as f:
    code = compile(f.read(), filename=str(script_path), mode='exec')
    exec(code, {"__file__": str(script_path)})

StopIteration: 

## Cálculo de la Potencia Radiactiva (FRP) en La Palma

Este bloque ejecuta el script `FRP_auto.py`, que forma parte del pipeline de procesamiento térmico del volcán de La Palma. Su objetivo es calcular la **Fire Radiative Power (FRP)** diaria, un indicador indirecto de actividad térmica en el área volcánica.

### Pasos que realiza el script:

1. **Carga del archivo mensual de Temperatura de Brillo (BT):**  
   Se abre el NetCDF correspondiente al mes actual, ubicado en `BT_daily_pixels`.

2. **Selección de la escena diaria:**  
   Se filtra la variable `BT_I05` correspondiente al día anterior (`datetime.now() - 1 día`).

3. **Aplicación de una máscara geográfica:**  
   Se limita el análisis a una región definida en torno al volcán (latitudes entre 28.54 y 28.57, longitudes entre -17.74 y -17.70).

4. **Cálculo de la temperatura media y la FRP:**  
   Usando la ley de Stefan-Boltzmann, se estima la potencia radiativa:

   \[
   \text{FRP} = \sigma \cdot (T^4 - T_{\text{floor}}^4) \cdot \text{área}
   \]


5. **Exportación del resultado:**  
   La FRP diaria válida se guarda en un archivo `radiative_power.nc`, organizado por fechas.

---

Este script permite llevar un seguimiento temporal de la evolución térmica del volcán, útil para detectar patrones estacionales o anomalías térmicas relacionadas con posibles cambios hidrotermales o actividad sísmica.

In [20]:
# === Notebook cell: Ejecutar FRP_auto.py de La Palma y documentar ===

import runpy
from pathlib import Path

# Paso 1: Localiza la ruta al script original de FRP de La Palma
project_root = next(p for p in Path().resolve().parents if "Practicas_Empresa_CSIC-1" in p.name)
script_path = project_root / "A01_source" / "B01_3_processing" / "La_Palma" / "radiative_power" / "RP_auto.py"

# Paso 2: Ejecuta el script como módulo principal (como si fuera run desde terminal)
resultados = runpy.run_path(str(script_path), run_name="__main__")




=== CALCULATING FRP FOR 2025-05-04 ===
2025-05-04 → BTmean=286.35 K, FRP=20.03 MW
✔︎ FRP appended to radiative_power.nc


/Users/moni/Desktop/Practicas_Empresa_CSIC-1/A01_source/B01_3_processing/La_Palma/radiative_power/RP_auto.py:64: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  total = (datetime.utcnow() - cutoff).days
